In [22]:
import torch
import torchvision.transforms as tf
import albumentations as A
from torchvision.transforms.functional import adjust_sharpness



# 0. Normal video cloning

In [23]:
import cv2

def read_and_write_video(input_video_path, output_video_path):
    """
    Reads a video from a file and writes it to another file in MP4 format.
    
    Args:
        input_video_path (str): Path to the input video file.
        output_video_path (str): Path to the output video file.
    """
    # Open the input video file
    cap = cv2.VideoCapture(input_video_path)
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))    
    # Create the video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    
    
    # Process each frame
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Write the frame to the output video
        out.write(frame)
        # Update the frame count
        frame_count += 1
    # Release the video capture and writer
    cap.release()
    out.release()
    # Get output video information
    out_cap = cv2.VideoCapture(output_video_path)
    out_cap.release()
    



# 1. Lower sharpness

In [24]:



def reduce_sharpness(input_video_path, output_video_path, sharpness_factor):
    """
    Reduces the sharpness of a video.
    
    Args:
        input_video_path (str): Path to the input video file.
        output_video_path (str): Path to the output video file.
        sharpness_factor (float, optional): Sharpness factor, ranging from 0 (blurred) to 2 (sharpened). Defaults to 0.5.
    """
    totensor = tf.ToTensor()
    # Load the video
    cap = cv2.VideoCapture(input_video_path)
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # Create the video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # correct format
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    
    
    
    # Process each frame
    frame_count = 0
    for frame_count in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        # Convert the frame to a PyTorch tensor
        frame_tensor = totensor(frame)
        # Adjust the sharpness of the frame 
        adjusted_frame_tensor = adjust_sharpness(frame_tensor, sharpness_factor)
        # Convert the adjusted frame back to a NumPy array
        adjusted_frame = (adjusted_frame_tensor.squeeze(0).permute(1, 2, 0) * 255).byte().numpy()
        # Write the adjusted frame to the output video
        out.write(adjusted_frame)
        frame_count += 1
        
    # Release the video capture and writer
    cap.release()
    out.release()
    
    # Print information about the output video after sharpness
    cv2.VideoCapture(output_video_path).release()
    cv2.destroyAllWindows()




In [28]:

reduce_sharpness('./videos/football.mp4','output/low_sharp.mp4', 60) # average sharpness augmentation time: 11s
# read_and_write_video('./videos/football.mp4','output/low_sharp.mp4')

# 2. Gaussian noise

In [29]:



def gaussian_noise(input_video_path, output_video_path, noise_std):
    """
    Add Gaussian noise to video using Albumentations
    
    Args:
        input_video_path (str): Path to the input video file.
        output_video_path (str): Path to the output video file.
        sharpness_factor (float, optional): Noise stdev, default 10
    """
    totensor = tf.ToTensor()
    # Load the video
    cap = cv2.VideoCapture(input_video_path)
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # Create the video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # correct format
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    noise_transform = A.GaussNoise(var_limit=(noise_std**2, noise_std**2), p=1.0)
    
    
    # Process each frame
    frame_count = 0
    for frame_count in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        # Add Gaussian noise to the frame
        noisy_frame = noise_transform(image=frame)['image']
        # Write the noisy frame to the output video
        out.write(noisy_frame)
        frame_count += 1
        
    # Release the video capture and writer
    cap.release()
    out.release()
    
    # Print information about the output video after sharpness
    cv2.VideoCapture(output_video_path).release()
    cv2.destroyAllWindows()




In [35]:
gaussian_noise('./videos/football.mp4','output/gaussian.mp4', 20)

# Defocus

In [32]:



def defocus(input_video_path, output_video_path, blur_limit=(3, 7), sigma_limit=0.5):
    """
    Add defocus blur to video using Albumentations
    
    Args:
        input_video_path (str): Path to the input video file.
        output_video_path (str): Path to the output video file.
        blur_limit (tuple, optional): Range of blur kernel size. Defaults to (3, 7).
        sigma_limit (float, optional): Maximum sigma value used for blurring the image. Defaults to 0.5.
    """
    # totensor = tf.ToTensor()
    # Load the video
    cap = cv2.VideoCapture(input_video_path)
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # Create the video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # correct format
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    defocus_transform = A.Blur(blur_limit=blur_limit, sigma_limit=sigma_limit, p=1.0)    
    
    # Process each frame
    frame_count = 0
    for frame_count in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        # Add Gaussian noise to the frame
        # noisy_frame = noise_transform(image=frame)['image']
        # Write the noisy frame to the output video
        # Apply Defocus Blur to the frame
        blurred_frame = defocus_transform(image=frame)['image']
        # Write the blurred frame to the output video
        out.write(blurred_frame)
        frame_count += 1
        
    # Release the video capture and writer
    cap.release()
    out.release()
    
    # Print information about the output video after sharpness
    cv2.VideoCapture(output_video_path).release()
    cv2.destroyAllWindows()




In [34]:
defocus('./videos/football.mp4','output/defocus.mp4')

/tmp/ipykernel_37362/3623384518.py:21: UserWarning: Argument 'sigma_limit' is not valid and will be ignored.
  defocus_transform = A.Blur(blur_limit=blur_limit, sigma_limit=sigma_limit, p=1.0)


# 4. Motion blur

In [3]:
import cv2
import albumentations as A


def motion_blur(input_video_path, output_video_path, kernel_size=(15, 15)):
    """
    Add motion blur  to video using Albumentations
    
    Args:
        input_video_path (str): Path to the input video file.
        output_video_path (str): Path to the output video file.
        kernel_size (tuple, optional): Size of the motion blur kernel. Defaults to (15, 15).
    """
    # totensor = tf.ToTensor()
    # Load the video
    cap = cv2.VideoCapture(input_video_path)
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # Create the video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # correct format
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    motion_blur_transform = A.MotionBlur(blur_limit=kernel_size, angle=angle, p=1.0)    
    
    # Process each frame
    frame_count = 0
    for frame_count in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        # Apply Motion Blur to the frame
        blurred_frame = motion_blur_transform(image=frame)['image']
        # Write the blurred frame to the output video
        out.write(blurred_frame)
        
        frame_count += 1
        
    # Release the video capture and writer
    cap.release()
    out.release()
    
    # Print information about the output video after sharpness
    cv2.VideoCapture(output_video_path).release()
    cv2.destroyAllWindows()




/home/longvu/anaconda3/envs/python3.10/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.2.2) or chardet (None)/charset_normalizer (3.3.2) doesn't match a supported version!
  warnings.warn(
/home/longvu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
motion_blur('./videos/football.mp4','output/motion_blur.mp4')

/tmp/ipykernel_72087/1781256144.py:25: UserWarning: Argument 'angle' is not valid and will be ignored.
  motion_blur_transform = A.MotionBlur(blur_limit=kernel_size, angle=angle, p=1.0)


# 5. Jitter

In [7]:
import cv2
import albumentations as A


def jitter(input_video_path, output_video_path, shift_limit=0.1, scale_limit=0.1, rotate_limit=45):
    """
    Add jitter to video using Albumentations
    
    Args:
        input_video_path (str): Path to the input video file.
        output_video_path (str): Path to the output video file.
        shift_limit (float, optional): Limit of horizontal and vertical shift. Defaults to 0.1.
        scale_limit (float, optional): Limit of scaling. Defaults to 0.1.
        rotate_limit (int, optional): Limit of rotation in degrees. Defaults to 45.
    """
    # totensor = tf.ToTensor()
    # Load the video
    cap = cv2.VideoCapture(input_video_path)
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # Create the video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # correct format
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    jitter_transform = A.Compose([
            A.RandomScale(scale_limit=scale_limit, p=1.0),
            A.RandomRotate90(p=1.0),
            A.Rotate(limit=rotate_limit, p=1.0),
            # A.RandomShift(shift_limit=shift_limit, p=1.0)
        ])    
    # Process each frame
    frame_count = 0
    for frame_count in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        # Apply Jitter to the frame
        jittered_frame = jitter_transform(image=frame)['image']
        # Write the jittered frame to the output video
        out.write(jittered_frame)
        
        frame_count += 1
        
    # Release the video capture and writer
    cap.release()
    out.release()
    
    # Print information about the output video after sharpness
    cv2.VideoCapture(output_video_path).release()
    cv2.destroyAllWindows()




In [9]:
jitter('./videos/football.mp4','output/jitter.mp4')